In [1]:
from ultralytics import YOLO
import cv2
# import pytesseract
import easyocr

model = YOLO("/media/dipen/New Volume/RnC Taskphase/OpenCV/best(1).pt")
# pytesseract.pytesseract.tesseract_cmd = r"C:\\Program Files\\Tesseract-OCR\\tesseract.exe"
classes = model.names
reader = easyocr.Reader(['en'], gpu=True)

# === 2. ROI extractor ===
def extract_scoreboard_roi(results):
    scoreboard_rois, time_rois, extra_time_rois, additional_info_rois = [], [], [], []
    for r in results:
        boxes = r.boxes.xyxy
        confs = r.boxes.conf
        clses = r.boxes.cls
        for box, conf, cls in zip(boxes, confs, clses):
            x1, y1, x2, y2 = box.tolist()
            name = model.names[int(cls)]
            if name == "Scoreboard":
                scoreboard_rois.append(((x1+7), (y1+7), (x2+7), (y2+7)))
            elif name == "time":
                time_rois.append(((x1+7), (y1+7), (x2+7), (y2+7)))
            elif name == "extra_time":
                extra_time_rois.append(((x1+7), (y1+7), (x2+7), (y2+7)))
            elif name == "additonalinfo":
                additional_info_rois.append(((x1+7), (y1+7), (x2+7), (y2+7)))
    return scoreboard_rois, time_rois, extra_time_rois, additional_info_rois

# === 3. OCR helper ===
# def OcrResult(img, roi_list):
#     if roi_list:
#         x1, y1, x2, y2 = map(int, roi_list[0])  # Take first ROI
#         roi_frame = img[y1:y2, x1:x2]
#         gray = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)
#         frame= cv2.Canny(gray, 50, 150)
#         _, thresh = cv2.threshold(frame, 180, 255, cv2.THRESH_BINARY)
#         cv2.imshow("ROI Frame", thresh)  # Show the ROI frame for debugging
#         text = pytesseract.image_to_string(thresh, config="--psm 7 --oem 3")
#         return text.strip()
# #     return ""
def OcrResult(img, roi_list):
    if roi_list:
        x1, y1, x2, y2 = map(int, roi_list[0])  # Take first ROI
        roi_frame = img[y1:y2, x1:x2]
        roi_rgb = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)
        # canny = cv2.Canny(roi_rgb, 50, 150)
        # _, thresh = cv2.threshold(canny, 150, 255, cv2.THRESH_BINARY)

        # EasyOCR detection
        results = reader.readtext(roi_rgb, detail=0)  # detail=0 returns just text strings

        if results:
            return " ".join(results).strip()
    return ""


# === 4. Process video ===
video_path = "/media/dipen/New Volume/RnC Taskphase/OpenCV/SampleVideo.mp4"
cap = cv2.VideoCapture(video_path)
i=0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO detection
    results = model(frame, verbose=False)

    # Extract ROIs
    scoreboard_roi, time_roi, extra_time_roi, additional_info_roi = extract_scoreboard_roi(results)
    # OCR
    scoreboard_text = OcrResult(frame, scoreboard_roi)
    time_text = OcrResult(frame, time_roi)
    extra_time_text = OcrResult(frame, extra_time_roi)
    additional_info_text = OcrResult(frame, additional_info_roi)
    i+=1
    # Show live results
    print(f"{(i)} Scoreboard: {scoreboard_text} | Time: {time_text} | Extra: {extra_time_text} | Info: {additional_info_text.strip()}")

    annotated_frame = results[0].plot()
    # annotated_frame.resize(640, 480)
    cv2.imshow("YOLO Detection", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1 Scoreboard: MUN Ukzu BOU | Time: 0i:20 | Extra:  | Info: 
2 Scoreboard: MUN U{ 0 BOU | Time: 0i26 | Extra:  | Info: 
3 Scoreboard: MUN U 0 BOU | Time: 07*26 | Extra:  | Info: 
4 Scoreboard: MUN 0 0 BOU | Time: 07*26 | Extra:  | Info: 
5 Scoreboard: MUN 0 0 BOU | Time: 0181 | Extra:  | Info: 
6 Scoreboard: MUN 0 0 BOU | Time: 0181 | Extra:  | Info: 
7 Scoreboard: MUN 0 0 BOU | Time: 0181 | Extra:  | Info: 
8 Scoreboard: MUN 0{ 0 BOU | Time: 0181 | Extra:  | Info: 
9 Scoreboard: MUN 0{ 0 BOU | Time: 0181 | Extra:  | Info: 
10 Scoreboard: MUN 0 0 BOU | Time: 0181 | Extra:  | Info: 
11 Scoreboard: MUN 0{ 0 BOU | Time: 0181 | Extra:  | Info: 
12 Scoreboard: MUN Ukzu BOU | Time: 0181 | Extra:  | Info: 
13 Scoreboard: MUN Ukzu BOU | Time: 0181 | Extra:  | Info: 
14 Scoreboard: MUN Ukzu BOU | Time: 0181 | Extra:  | Info: 
15 Scoreboard: MUN U 0 BOU | Time: 0181 | Extra:  | Info: 
16 Scoreboard: MUN 0t 0 BOU | Time: 0181 | Extra:  | Info: 
17 Scoreboard: MUN 0 0 BOU | Time: 01827 | Extra:  | 

In [4]:
from ultralytics import YOLO
import cv2
import easyocr
import time
import numpy as np

model = YOLO("/media/dipen/New Volume/RnC Taskphase/OpenCV/best(1).pt")
classes = model.names
reader = easyocr.Reader(['en'], gpu=True)
window=np.zeros((100, 250, 3), np.uint8)

PROCESS_EVERY_N_FRAMES = 3 
OCR_EVERY_N_FRAMES = 6      
frame_count = 0
last_scoreboard_text = ""
last_time_text = ""
last_extra_time_text = ""
last_additional_info_text = ""

# === 2. ROI extractor ===
def extract_scoreboard_roi(results):
    scoreboard_rois, time_rois, extra_time_rois, additional_info_rois = [], [], [], []
    for r in results:
        boxes = r.boxes.xyxy
        confs = r.boxes.conf
        clses = r.boxes.cls
        for box, conf, cls in zip(boxes, confs, clses):
            x1, y1, x2, y2 = box.tolist()
            name = model.names[int(cls)]
            if name == "Scoreboard":
                scoreboard_rois.append(((x1+7), (y1+7), (x2+7), (y2+7)))
            elif name == "time":
                time_rois.append(((x1+7), (y1+7), (x2+7), (y2+7)))
            elif name == "extra_time":
                extra_time_rois.append(((x1+7), (y1+7), (x2+7), (y2+7)))
            elif name == "additonalinfo":
                additional_info_rois.append(((x1+7), (y1+7), (x2+7), (y2+7)))
    return scoreboard_rois, time_rois, extra_time_rois, additional_info_rois

# === 3. OCR helper ===
def OcrResult(img, roi_list):
    if roi_list:
        x1, y1, x2, y2 = map(int, roi_list[0])  
        roi_frame = img[y1:y2, x1:x2]
        roi_rgb = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)
        results = reader.readtext(roi_rgb, detail=0)  
        if results:
            return " ".join(results).strip()
    return ""
def display_scoreboard(score, match_time, extra_time, info):
    h, w = 300, 600
    scoreboard_img = np.zeros((h, w, 3), dtype=np.uint8)  # black background

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    color = (255, 255, 255)  
    thickness = 2
    cv2.putText(scoreboard_img, f"Score: {score}", (20, 50), font, font_scale, color, thickness)
    cv2.putText(scoreboard_img, f"Time: {match_time}", (20, 100), font, font_scale, color, thickness)
    cv2.putText(scoreboard_img, f"Extra Time: {extra_time}", (20, 150), font, font_scale, color, thickness)
    cv2.putText(scoreboard_img, f"Info: {info}", (20, 200), font, font_scale, color, thickness)
    cv2.imshow("Scoreboard", scoreboard_img)

# === 4. Process video ===
video_path = "/media/dipen/New Volume/RnC Taskphase/OpenCV/SourceVideos/prediction of big scoreboard.mp4"
cap = cv2.VideoCapture(video_path)

# Set buffer size to reduce latency
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

# Variables for performance tracking
start_time = time.time()
processed_frames = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_count += 1
    processed_frames += 1
    
    # Only run YOLO detection every N frames
    if frame_count % PROCESS_EVERY_N_FRAMES == 0:
        # Run YOLO detection
        results = model(frame, verbose=False)
        
        # Extract ROIs
        scoreboard_roi, time_roi, extra_time_roi, additional_info_roi = extract_scoreboard_roi(results)
        
        # Only run OCR every OCR_EVERY_N_FRAMES frames and if ROIs exist
        if frame_count % OCR_EVERY_N_FRAMES == 0:
            if scoreboard_roi:
                last_scoreboard_text = OcrResult(frame, scoreboard_roi)
            if time_roi:
                last_time_text = OcrResult(frame, time_roi)
            if extra_time_roi:
                last_extra_time_text = OcrResult(frame, extra_time_roi)
            if additional_info_roi:
                last_additional_info_text = OcrResult(frame, additional_info_roi)
        
        # Use last known results for display
        annotated_frame = results[0].plot()
        
        # Print results less frequently to reduce I/O overhead
        if frame_count % OCR_EVERY_N_FRAMES == 0:
            print(f"{frame_count} Scoreboard: {last_scoreboard_text} | Time: {last_time_text} | Extra: {last_extra_time_text} | Info: {last_additional_info_text}")
    else:
        annotated_frame = frame.copy()
    
    # Resize frame for faster display (optional - uncomment if needed)
    annotated_frame = cv2.resize(annotated_frame, (640, 480))
    display_scoreboard(last_scoreboard_text, last_time_text, last_extra_time_text, last_additional_info_text)
    cv2.imshow("Frame", annotated_frame)

    
    # Calculate and display FPS every 30 frames
    if processed_frames % 30 == 0:
        elapsed = time.time() - start_time
        fps = processed_frames / elapsed
        print(f"Current FPS: {fps:.2f}")
    
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

total_time = time.time() - start_time
final_fps = processed_frames / total_time
print(f"Average FPS: {final_fps:.2f}")
print(f"Total frames processed: {processed_frames}")
print(f"Total time: {total_time:.2f} seconds")

Average FPS: 0.40
Total frames processed: 1
Total time: 2.48 seconds


In [1]:
import cv2
import numpy as np
import time
from ultralytics import YOLO
import easyocr
teams=["BOU", "LIV", "MCI", "MUN", "TOT", "CHE", "ARS", "EVE", "WOL", "CRY",
       "NEW", "AVL", "BRE", "BHA", "BRN", "FUL", "LEE", "NFO", "SUN", "WHU"]

# ==== Centered text drawing ====
def draw_centered_text(img, text, box_coords, font, scale, color, thickness):
    (x1, y1), (x2, y2) = box_coords
    text_size = cv2.getTextSize(text, font, scale, thickness)[0]
    text_width, text_height = text_size
    x = x1 + (x2 - x1 - text_width) // 2
    y = y1 + (y2 - y1 + text_height) // 2
    cv2.putText(img, text, (x, y), font, scale, color, thickness)

# ==== Draw live scoreboard ====
def display_scoreboard(team1, team2, score, match_time, extra_time, info):
    h, w = 300, 500
    img = np.zeros((h, w, 3), dtype=np.uint8)
    white = (255, 255, 255)
    red = (0, 0, 255)
    black = (0, 0, 0)

    # Background
    cv2.rectangle(img, (0, 0), (w, h), white, -1)

    # Layout lines
    cv2.line(img, (0, 90), (w, 90), black, 2)      # after team row
    cv2.line(img, (w//2, 0), (w//2, 90), black, 2) # between team boxes
    cv2.line(img, (0, 200), (w, 200), black, 2)    # after score row
    cv2.line(img, (0, 250), (w, 250), black, 2)    # after time row

    # Draw centered text
    draw_centered_text(img, team1, ((0, 0), (w//2, 90)), cv2.FONT_HERSHEY_SIMPLEX, 1, black, 2)
    draw_centered_text(img, team2, ((w//2, 0), (w, 90)), cv2.FONT_HERSHEY_SIMPLEX, 1, black, 2)
    draw_centered_text(img, score, ((0, 90), (w, 200)), cv2.FONT_HERSHEY_SIMPLEX, 3, red, 3)
    draw_centered_text(img, f"{match_time} {extra_time}", ((0, 200), (w, 250)), cv2.FONT_HERSHEY_SIMPLEX, 1, red, 2)
    draw_centered_text(img, f"Info: {info}", ((0, 250), (w, h)), cv2.FONT_HERSHEY_SIMPLEX, 1, black, 2)

    cv2.imshow("Match Details", img)

# ==== YOLO + OCR setup ====
model = YOLO("/media/dipen/New Volume/RnC Taskphase/OpenCV/best (1).pt")
reader = easyocr.Reader(['en'], gpu=True)

PROCESS_EVERY_N_FRAMES = 3
OCR_EVERY_N_FRAMES = 6
frame_count = 0

scoreboard=[]
last_scoreboard_text = ""
last_time_text = ""
last_extra_time_text = ""
last_additional_info_text = ""

score_team1 = 0
score_team2 = 0

# === ROI extractor ===
def extract_scoreboard_roi(results):
    scoreboard_rois, time_rois, extra_time_rois, additional_info_rois = [], [], [], []
    for r in results:
        boxes = r.boxes.xyxy
        clses = r.boxes.cls
        for box, cls in zip(boxes, clses):
            x1, y1, x2, y2 = box.tolist()
            name = model.names[int(cls)]
            if name == "Scoreboard":
                scoreboard_rois.append((x1+7, y1+7, x2+7, y2+7))
            elif name == "time":
                time_rois.append((x1+7, y1+7, x2+7, y2+7))
            elif name == "extra_time":
                extra_time_rois.append((x1+7, y1+7, x2+7, y2+7))
            elif name == "additonalinfo":
                additional_info_rois.append((x1+7, y1+7, x2+7, y2+7))
    return scoreboard_rois, time_rois, extra_time_rois, additional_info_rois

# === OCR helper ===
def OcrResult(img, roi_list):
    if roi_list:
        x1, y1, x2, y2 = map(int, roi_list[0])
        roi_frame = img[y1:y2, x1:x2]
        roi_gray = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)
        canny = cv2.Canny(roi_gray, 50, 150)
        _, thresh = cv2.threshold(canny, 150, 255, cv2.THRESH_BINARY)
        results = reader.readtext(thresh, detail=0)
        if results:
            return " ".join(results).strip()
    return ""

# ==== Video processing ====
video_path = "/media/dipen/New Volume/RnC Taskphase/OpenCV/SampleVideo.mp4"
cap = cv2.VideoCapture(video_path)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

start_time = time.time()
processed_frames = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    processed_frames += 1

    if frame_count % PROCESS_EVERY_N_FRAMES == 0:
        results = model(frame, verbose=False)
        scoreboard_roi, time_roi, extra_time_roi, additional_info_roi = extract_scoreboard_roi(results)

        if frame_count % OCR_EVERY_N_FRAMES == 0:
            if scoreboard_roi:
                last_scoreboard_text = OcrResult(frame, scoreboard_roi)
                scoreboard.append(OcrResult(frame, scoreboard_roi))
                for team in teams:
                    if team.lower() in last_scoreboard_text.lower():
                        if "goal" in last_scoreboard_text.lower():
                            if team == "Team A":
                                score_team1 += 1
                            elif team == "Team B":
                                score_team2 += 1
                # if "goal" in last_scoreboard_text.lower() :
                #     score_team1 += 1 
                    
            if time_roi:
                last_time_text = OcrResult(frame, time_roi)
            if extra_time_roi:
                last_extra_time_text = OcrResult(frame, extra_time_roi)
            if additional_info_roi:
                last_additional_info_text = OcrResult(frame, additional_info_roi)

        annotated_frame = results[0].plot()
        if frame_count % OCR_EVERY_N_FRAMES == 0:
            print(f"{frame_count} Scoreboard: {last_scoreboard_text} | Time: {last_time_text} | Extra: {last_extra_time_text} | Info: {last_additional_info_text}")
    else:
        annotated_frame = frame.copy()

    annotated_frame = cv2.resize(annotated_frame, (640, 480))

    # Show both annotated frame & custom scoreboard
    display_scoreboard("Team A", "Team B", last_scoreboard_text, last_time_text, last_extra_time_text, last_additional_info_text)
    cv2.imshow("Frame", annotated_frame)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

total_time = time.time() - start_time
print(f"Average FPS: {processed_frames / total_time:.2f}")
print(f"Total frames processed: {processed_frames}")
print(f"Total time: {total_time:.2f} seconds")


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/dipen/.local/lib/python3.10/site-packages/cv2/qt/plugins"


6 Scoreboard: NUN-(} (0< Bou UcCni | Time: @1732,7 TieTcriin Oot | Extra:  | Info: 
12 Scoreboard: MUN-( _ BOu Jk#aIs | Time: (732,7 DelrTIAt Ln | Extra:  | Info: 
18 Scoreboard: {MUN-(0 BO0 4"nke | Time: @Z;27 TLrcTHDA Lanatoln | Extra:  | Info: 
24 Scoreboard: MUN-(0l 0) BOUU | Time: @7327 Yietoiotreeerorao | Extra:  | Info: 
30 Scoreboard: MUN-(0 (0) BOQU | Time: @7a27 ULEIANE Lrteua | Extra:  | Info: 
36 Scoreboard: MUN-(l (0S BOu | Time: @07328 Aalans | Extra:  | Info: 
42 Scoreboard: MUN 06300 BU | Time: 1Z 522085 oTC rttl 4tl | Extra:  | Info: 
48 Scoreboard: MUN  BoU EAn | Time: 02a288 36na | Extra:  | Info: 
54 Scoreboard: MUN- BOU | Time: @7328 EMIp | Extra:  | Info: 
60 Scoreboard: MUN 0 BOU | Time: 07a28 | Extra:  | Info: 
66 Scoreboard: MUN-(LS(0 BOU | Time: 07a29 | Extra:  | Info: 
72 Scoreboard: MUN-0 (0) BOU | Time: @7;2@ 43049u M | Extra:  | Info: 
78 Scoreboard: MUN (l_(<Bou} | Time: 07a2@ | Extra:  | Info: 
84 Scoreboard: MUN-{l (0) BOU | Time: GZa2@ Eac | Extra:  | 

In [1]:
import cv2
import numpy as np
import time
from ultralytics import YOLO
import easyocr

model = YOLO("D:\RnC Taskphase\OpenCV\\best(1).pt")
reader = easyocr.Reader(['en'], gpu=True) 

PROCESS_EVERY_N_FRAMES = 3
OCR_EVERY_N_FRAMES = 3
frame_count = 0
cooldown_seconds = 8
last_goal_time = 0

scoreboard = []
time_history = []
extra_time_history = []
info_history = []
last_scoreboard_text = ""
last_time_text = ""
last_extra_time_text = ""
last_additional_info_text = ""
hometeam=""
awayteam=""
home_score = 0
away_score = 0
teams = ["BOU", "LIV", "MCI", "MUN", "TOT", "CHE", "ARS", "EVE", "WOL", "CRY",
         "NEW", "AVL", "BRE", "BHA", "BRN", "FUL", "LEE", "NFO", "SUN", "WHU"]
h, w = 300, 500
img = np.zeros((h, w, 3), dtype=np.uint8)

# ==== Centered text drawing ====
def draw_centered_text(img, text, box_coords, font, scale, color, thickness):
    text = "" if text is None else str(text)
    (x1, y1), (x2, y2) = box_coords
    text_size = cv2.getTextSize(text, font, scale, thickness)[0]
    text_width, text_height = text_size
    x = x1 + (x2 - x1 - text_width) // 2
    y = y1 + (y2 - y1 + text_height) // 2
    cv2.putText(img, text, (x, y), font, scale, color, thickness, lineType=cv2.LINE_AA)

# ==== Draw live scoreboard ====
def display_scoreboard(team1, team2, score, match_time, extra_time, info):
    white, red, black = (255, 255, 255), (0, 0, 255), (0, 0, 0)

    cv2.rectangle(img, (0, 0), (w, h), white, -1)

    # Layout lines
    cv2.line(img, (0, 90), (w, 90), black, 2)
    cv2.line(img, (w//2, 0), (w//2, 90), black, 2)
    cv2.line(img, (0, 200), (w, 200), black, 2)
    cv2.line(img, (0, 250), (w, 250), black, 2)

    # Centered text
    draw_centered_text(img, team1, ((0, 0), (w//2, 90)), cv2.FONT_HERSHEY_SIMPLEX, 1, black, 2)
    draw_centered_text(img, team2, ((w//2, 0), (w, 90)), cv2.FONT_HERSHEY_SIMPLEX, 1, black, 2)
    draw_centered_text(img, str(score), ((0, 90), (w, 200)), cv2.FONT_HERSHEY_SIMPLEX, 3, red, 3)
    draw_centered_text(img, f"{str(match_time)} {str(extra_time)}".strip(), ((0, 200), (w, 250)), cv2.FONT_HERSHEY_SIMPLEX, 1, red, 2)
    draw_centered_text(img, f"Info: {str(info)}", ((0, 250), (w, h)), cv2.FONT_HERSHEY_SIMPLEX, 1, black, 2)

    return img

# === ROI extractor ===
def extract_scoreboard_roi(results):
    scoreboard_rois, time_rois, extra_time_rois, additional_info_rois = [], [], [], []
    for r in results:
        boxes = r.boxes.xyxy
        clses = r.boxes.cls
        for box, cls in zip(boxes, clses):
            x1, y1, x2, y2 = box.tolist()
            name = model.names[int(cls)]
            if name == "Scoreboard":
                scoreboard_rois.append((x1+7, y1+7, x2+7, y2+7))
            elif name == "time":
                time_rois.append((x1+7, y1+7, x2+7, y2+7))
            elif name == "extra_time":
                extra_time_rois.append((x1+7, y1+7, x2+7, y2+7))
            elif name == "additonalinfo":
                additional_info_rois.append((x1+7, y1+7, x2+7, y2+7))
    return scoreboard_rois, time_rois, extra_time_rois, additional_info_rois

# === OCR helper ===
def OcrResult(img, roi_list):
    if roi_list:
        x1, y1, x2, y2 = map(int, roi_list[0])
        roi_frame = img[y1:y2, x1:x2]
        roi_gray = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2RGB)
        # _, thresh = cv2.threshold(roi_gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        results = reader.readtext(roi_gray, detail=0)
        if results:
            return " ".join(results).strip()
    return ""
def ScoreDetect(scoreboard_detected,hometeam,awayteam,home_score,away_score):
    global last_goal_time
    detected_text=scoreboard_detected
    if "GOAL" in detected_text:
        now = time.time()
        if now - last_goal_time > cooldown_seconds:  # only count new goals after cooldown
            print("⚽ Goal detected!")
            last_goal_time = now

            if hometeam != "" and hometeam in detected_text:
                home_score += 1
            elif awayteam != "" and awayteam in detected_text:
                away_score += 1
    return home_score, away_score


# ==== Video processing ====
video_path = "D:\RnC Taskphase\OpenCV\SourceVideos\\bigScoreboard.mp4"
cap = cv2.VideoCapture(video_path)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

start_time = time.time()
processed_frames = 0

while True:
    ret, frame = cap.read()
    ret1, frame1 = cap.read()
    if not ret:
        break
    x1, y1 = 175,40
    x2, y2 = 205,80
    cv2.rectangle(frame, (x1,y1),(x2,y2), (0, 0, 0), -1)
    frame_count += 1
    processed_frames += 1

    if frame_count % PROCESS_EVERY_N_FRAMES == 0:
        results = model(frame, verbose=False)
        scoreboard_roi, time_roi, extra_time_roi, additional_info_roi = extract_scoreboard_roi(results)

        if frame_count % OCR_EVERY_N_FRAMES == 0:
            if scoreboard_roi:
                detected_score = OcrResult(frame, scoreboard_roi)
                last_scoreboard_text = "" if detected_score is None else str(detected_score)

                half_index = len(detected_score) // 2
                left_side = detected_score[:half_index].strip()
                right_side = detected_score[half_index:].strip()

                for team in teams:
                    if team in left_side and hometeam == "":
                        hometeam = team
                    if team in right_side and awayteam == "":
                        awayteam = team
                home_score,away_score=ScoreDetect(detected_score,hometeam,awayteam,home_score,away_score)

            if time_roi:
                detected_time = OcrResult(frame, time_roi)
                last_time_text = "" if detected_time is None else str(detected_time)

            if extra_time_roi:
                detected_extra = OcrResult(frame, extra_time_roi)
                last_extra_time_text = "" if detected_extra is None else str(detected_extra)

            if additional_info_roi:
                detected_info = OcrResult(frame, additional_info_roi)
                last_additional_info_text = "" if detected_info is None else str(detected_info)

            print(f"{frame_count} Scoreboard: {last_scoreboard_text} | Time: {last_time_text} | Extra: {last_extra_time_text} | Info: {last_additional_info_text}")

        annotated_frame = results[0].plot()
    else:
        annotated_frame = frame.copy()

    # annotated_frame = cv2.resize(annotated_frame, (640, 480))

    scoreboard_img = display_scoreboard(hometeam, awayteam, f"{home_score}-{away_score}",last_time_text, last_extra_time_text, last_additional_info_text)
    cv2.imshow("Match Details", scoreboard_img)
    cv2.imshow("Frame", annotated_frame)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

scoreboard = []
time_history = []
extra_time_history = []
info_history = []
last_scoreboard_text = ""
last_time_text = ""
last_extra_time_text = ""
last_additional_info_text = ""
hometeam=""
awayteam=""
home_score = 0
away_score = 0

total_time = time.time() - start_time
print(f"Average FPS: {processed_frames / total_time:.2f}")
print(f"Total frames processed: {processed_frames}")
print(f"Total time: {total_time:.2f} seconds")

print(f"Final Score: {hometeam} {home_score} - {away_score} {awayteam}")


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
C:\Users\Dipen B\AppData\Roaming\Python\Python310\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


3 Scoreboard: Wolves 0) chelsea | Time:  | Extra:  | Info: First Half
6 Scoreboard: Wolves 0) chelsea | Time:  | Extra:  | Info: First Half
9 Scoreboard: Wolves 1 6, chelsea | Time:  | Extra:  | Info: First Half
12 Scoreboard: Wolves 0 0 chelsea | Time:  | Extra:  | Info: First Half
15 Scoreboard: Wolves _ 0 chelsea | Time:  | Extra:  | Info: First Half
18 Scoreboard: Wolves 1, 0) Chelsea | Time:  | Extra:  | Info: First Half
21 Scoreboard: Wolves 1, 0) Chelsea | Time:  | Extra:  | Info: First Half
24 Scoreboard: Wolves 1, 0) Chelsea | Time:  | Extra:  | Info: First Half
27 Scoreboard: Wolves 1, 0) Chelsea | Time:  | Extra:  | Info: First Half
30 Scoreboard: Wolves 1, 0) Chelsea | Time:  | Extra:  | Info: First Half
33 Scoreboard: Wolves 1, 0) Chelsea | Time:  | Extra:  | Info: First Half
36 Scoreboard:  | Time:  | Extra:  | Info: First Half
39 Scoreboard: WOL 0 CHE | Time: U:1/ | Extra:  | Info: AobLO
42 Scoreboard: WOL 0 CHE | Time: Uu:1Z | Extra:  | Info: HUBLOT
45 Scoreboard: WOL 0